# 📊 Customer Reviews Sentiment Analysis (SQL + Python + NLTK)

This notebook performs sentiment analysis on customer reviews stored in a SQL Server database.  
We use Python tools like `pandas`, `pyodbc`, and `nltk` (VADER) to analyze textual sentiment, combine it with rating data, and classify reviews for business insights.

In [ ]:
# 📦 Install required packages
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# pip install pandas nltk pyodbc sqlalchemy

import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\FAZA\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## 🔌 Connect to SQL Server and Fetch Data
Define the connection string and retrieve customer reviews data from the SQL Server database.

In [7]:
def fetch_data_from_sql():
    conn_str = (
        "Driver={SQL Server};"
        "Server=LAPTOP-8JQM3T7N\\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# Load data
customer_reviews_df = fetch_data_from_sql()
customer_reviews_df.head()

C:\Users\FAZA\AppData\Local\Temp\ipykernel_5760\1015474308.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


## 🧠 Sentiment Analysis with VADER
We'll calculate a **compound sentiment score** from the review text using the VADER analyzer.

In [8]:
# Initialize VADER
sia = SentimentIntensityAnalyzer()

# Function to get compound score
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

# Apply sentiment analysis
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)
customer_reviews_df[['ReviewText', 'SentimentScore']].head()

,ReviewText,SentimentScore
0,"Average experience, nothing special.",-0.3089
1,The quality is top-notch.,0.0000
2,Five stars for the quick delivery.,0.0000
3,"Good quality, but could be cheaper.",0.2382
4,"Average experience, nothing special.",-0.3089


## 🏷️ Categorize Sentiment using Rating and Text
Classify reviews into: Positive, Negative, Mixed, or Neutral.

In [9]:
# Define a function to categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment
        
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

## 📊 Sentiment Score Buckets
Bucket the sentiment score into ranges for clearer segmentation.

In [11]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment
    
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

## 📌 Final Output Preview
Let's preview the DataFrame with sentiment score, category, and bucket.

In [12]:
customer_reviews_df[['ReviewText', 'Rating', 'SentimentScore', 'SentimentCategory', 'SentimentBucket']].head(10)

,ReviewText,Rating,SentimentScore,SentimentCategory,SentimentBucket
0,"Average experience, nothing special.",3,-0.3089,Mixed Negative,-0.49 to 0.0
1,The quality is top-notch.,5,0.0000,Positive,0.0 to 0.49
2,Five stars for the quick delivery.,4,0.0000,Positive,0.0 to 0.49
3,"Good quality, but could be cheaper.",3,0.2382,Mixed Positive,0.0 to 0.49
4,"Average experience, nothing special.",3,-0.3089,Mixed Negative,-0.49 to 0.0
5,Customer support was very helpful.,4,0.6997,Positive,0.5 to 1.0
6,"Average experience, nothing special.",3,-0.3089,Mixed Negative,-0.49 to 0.0
7,The quality is top-notch.,5,0.0000,Positive,0.0 to 0.49
8,"I love this product, will buy again!",4,0.6696,Positive,0.5 to 1.0
9,"Excellent product, highly recommend!",5,0.7773,Positive,0.5 to 1.0


## 💾 Save Output to CSV
Export the enhanced data for further use in dashboards or reports.

In [14]:
# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index=False)
print("✅ File saved successfully.")

✅ File saved successfully.
